In [1]:
import os
import json
import h5py
import numpy as np

import robomimic
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.obs_utils as ObsUtils
import imageio
import tqdm
from robomimic.utils.file_utils import create_hdf5_filter_key

In [2]:
dataset_path = "/home/ns/collect_robomimic_demos/PickPlaceCan_02_09_2024_04_37PM_sojib/demo_image.hdf5" 
f_org = h5py.File(dataset_path, "r")
demos = list(f_org["data"].keys())

In [3]:
lengths=[]
for demo_name in demos:
    demo=f_org['data'][demo_name]
    num_samples=demo.attrs['num_samples']
    lengths.append(num_samples)

lengths=np.array(lengths)

print('Number of demos: ', len(demos))
print('Max length: ', np.max(lengths))
print('Min length: ', np.min(lengths))
print('Mean length: ', np.mean(lengths))

Number of demos:  65
Max length:  363
Min length:  182
Mean length:  261.4


In [4]:
demos[:3]

['demo-True_1', 'demo-True_10', 'demo-True_11']

In [5]:
len(demos)

65

In [6]:
f_org.close()

In [5]:
# demos2copy=[demo.decode('utf-8') for demo in f_org['mask']['expert']] 
# # demos2copy=demos

demos2copy=['demo-True_7', 'demo-True_16']
len(demos2copy)

2

In [8]:
demos2copy[:5]

['demo_1', 'demo_10', 'demo_100', 'demo_11', 'demo_12']

In [6]:
fn='/home/ns/collect_robomimic_demos/PickPlaceCan_02_09_2024_04_37PM_sojib/videos/expert/expert2.txt'
with open(fn, 'r') as f:
    file_demos = f.readlines()
demos = [x.strip().replace(".mp4","").strip() for x in file_demos]
len(demos), demos[:5]

(50,
 ['demo-True_10',
  'demo-True_11',
  'demo-True_18',
  'demo-True_19',
  'demo-True_1'])

In [7]:
demos2copy=demos

In [8]:
dataset_path_sub =dataset_path
dataset_path_sub=dataset_path_sub.replace(".hdf5", "")+"_sub.hdf5"
dataset_path_sub

'/home/ns/collect_robomimic_demos/PickPlaceCan_02_09_2024_04_37PM_sojib/demo_image_sub.hdf5'

In [9]:
f_sub = h5py.File(dataset_path_sub, "w")

In [10]:
keys_to_copy=demos2copy
len(keys_to_copy)

50

In [11]:
f_sub.create_group("data")
f_sub.create_group("mask")

<HDF5 group "/mask" (0 members)>

In [12]:
attrs=f_org["data"].attrs["env_args"]
f_sub["data"].attrs["env_args"] = attrs

In [13]:
i=1
for demo_name in tqdm.tqdm(keys_to_copy):
    b_demo=f_org['data'][demo_name]
    num_samples=b_demo.attrs['num_samples']
    model_file=b_demo.attrs['model_file']

    bdemo_name="demo_"+str(i)
    i = i+1

    f_sub["data"].create_group(bdemo_name)


    ks=['actions', 'dones', 'rewards', 'states']
    for k in ks:
        kdata = b_demo[k]
        f_sub["data"][bdemo_name].create_dataset(k, data=kdata)

    obs_keys =b_demo['obs'].keys()
    next_obs_keys=b_demo['next_obs'].keys()
    for k in obs_keys:
        kdata = b_demo['obs'][k]
        f_sub["data"][bdemo_name].create_dataset(f'obs/{k}', data=kdata)
    for k in next_obs_keys:
        kdata = b_demo['next_obs'][k]
        f_sub["data"][bdemo_name].create_dataset(f'next_obs/{k}', data=kdata)

    f_sub["data"][bdemo_name].attrs["num_samples"] = num_samples
    f_sub["data"][bdemo_name].attrs["model_file"] = model_file
 
# f_sub.close()
f_org.close()

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [00:03<00:00, 13.86it/s]


In [14]:
demos=demos2copy
demos= np.array(demos, dtype='S16') 
demos.shape, demos[:4]

((40,),
 array([b'demo-False_10', b'demo-False_12', b'demo-False_14',
        b'demo-False_15'], dtype='|S16'))

In [13]:
hdf5_path=dataset_path_sub 

filter_keys=sorted([elem for elem in demos])
filter_name="expert"
filter_lengths = create_hdf5_filter_key(hdf5_path=hdf5_path, demo_keys=filter_keys, key_name=filter_name)

In [14]:
f_sub.close()
f_org.close()